<a href="https://colab.research.google.com/github/Frederik-Roeckle/Rep_ImpactProcessComplexity/blob/master/Only_industry_variable_as_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import statsmodels.api as sm

In [85]:
drive.mount('/content/drive')

data = 'merged.csv'

# Freddys google drive path
data = '/content/drive/MyDrive/Colab Notebooks/Rep_ProcessMining/provided/merged.csv'

data = pd.read_csv(data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
column_indices = [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
                  21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
                  41, 42, 43, 44, 45, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54]

# Select the columns
datas_filtered = datas.iloc[:, column_indices]

In [87]:
import pandas as pd
import numpy as np

# Check for missing values
null_columns = datas_filtered.columns[datas_filtered.isnull().any()].tolist()
print("Columns with null values:")
print(null_columns)

# Check for infinite values
#inf_columns = datas_filtered.columns[np.isinf(datas_filtered).any()].tolist()
#print("Columns with infinite values:")
#print(inf_columns)


Columns with null values:
['Number of participants in period', 'Average number of participants per trace', 'Median number of participants per trace', 'Handover of work in period', 'Average handover of work per trace', 'Median handover of work per trace', 'Average handover of work from participant', 'Median handover of work from participant', 'Affinity']


### There are null values in some columns

In [88]:
# Drop rows where 'Affinity' is null
datas_filtered = datas_filtered.dropna(subset=['Affinity', 'Number of participants in period', 'Handover of work in period',
                                               'Median handover of work from participant', 'Average handover of work from participant',
                                               'Median handover of work per trace', 'Average handover of work per trace', 'Median number of participants per trace',
                                               'Average number of participants per trace'])

# Now 'df' is your dataframe with rows dropped where 'Affinity' is null

**FULL** **MODELS**


In [89]:
# Define the dependent variable (median throughput time)
y = datas_filtered["Median throughput time"]

# Define the independent variables (all other metrics)
X = datas_filtered.drop("Median throughput time", axis=1)

y

0         572551.5
1         638583.0
2         606969.0
3         694697.0
4         786880.0
           ...    
1424    48098190.0
1425    66773640.0
1426    66773640.0
1427    66773640.0
1428    66773640.0
Name: Median throughput time, Length: 1363, dtype: float64

**ADDING** **THE** **DUMMMY** **VARIABLE**

In [90]:
def assign_value(y):
    if y == ('BPIC15_3_metrics.csv'or 'BPIC15_4_metrics.csv' or 'BPIC15_5_metrics.csv' or 'BPIC15_2_metrics.csv' or 'BPIC15_1_metrics.csv' or 'BPI_Challenge_2018_metrics.csv'):  # replace this condition with your own
        return 'Public Administration'
    if y == ('2020_PrepaidTravelCost_metrics.csv' or '2020_DomesticDeclarations_metrics.csv' or '2020_RequestForPayment_metrics.csv' or '2020_InternationalDeclarations_metrics.csv' or '2020_PermitLog_metrics.csv'):
        return 'Education'
    if y == 'Hospital_log_metrics.csv':
      return 'Healthcare'
    if y == 'BPI_Challenge_2017_metrics.csv':
      return 'Finance'
    if y == 'BPI_Challenge_2019_metrics.csv':
      return 'Manufacturing'

dummy_filtered = datas.copy()

# Apply the function to the 'y' column and assign the result to a new column 'x'
dummy_filtered['INDUSTRY_DUMMY_VARIABLE'] = datas['File'].apply(assign_value)

In [91]:
dummy_filtered['INDUSTRY_DUMMY_VARIABLE'].value_counts()

INDUSTRY_DUMMY_VARIABLE
Manufacturing            868
Public Administration     63
Healthcare                39
Education                 26
Finance                   14
Name: count, dtype: int64

In [92]:
dummy_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474 entries, 0 to 1473
Data columns (total 57 columns):
 #   Column                                                             Non-Null Count  Dtype  
---  ------                                                             --------------  -----  
 0   Date                                                               1474 non-null   object 
 1   Cases                                                              1474 non-null   int64  
 2   Distinct activities                                                1474 non-null   object 
 3   Number of distinct activities                                      1474 non-null   int64  
 4   Number of simple cycles in period                                  1474 non-null   int64  
 5   Average number of simple cycles per trace                          1474 non-null   float64
 6   Median number of simple cycles per trace                           1474 non-null   float64
 7   Number of activity repet

In [93]:
column_indices = [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
                  21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
                  41, 42, 43, 44, 45, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56]

# Select the columns
dummy_filtered = dummy_filtered.iloc[:, column_indices]

dummy_filtered = dummy_filtered.dropna(subset=['Affinity', 'Number of participants in period', 'Handover of work in period',
                                               'Median handover of work from participant', 'Average handover of work from participant',
                                               'Median handover of work per trace', 'Average handover of work per trace', 'Median number of participants per trace',
                                               'Average number of participants per trace'])


In [94]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()

dummy_filtered['INDUSTRY_DUMMY_VARIABLE'] = enc.fit_transform(dummy_filtered['INDUSTRY_DUMMY_VARIABLE'])

In [95]:
y = dummy_filtered["Median throughput time"]

# Define the independent variables (all other metrics)
X = dummy_filtered["INDUSTRY_DUMMY_VARIABLE"]


**FULL MODELS WITH DUMMY VARIABLE**

In [96]:
model = sm.OLS(y,X).fit()

In [97]:
model.rsquared

0.26560601353812285